In [1]:

!pip install ultralytics roboflow torch matplotlib opencv-python 


  Using cached roboflow-1.1.60-py3-none-any.whl.metadata (9.7 kB)
  Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
Using cached roboflow-1.1.60-py3-none-any.whl (84 kB)
Using cached opencv_python_headless-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
   ---------------------------------------- 0.0/8.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.6 MB 4.2 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/8.6 MB 4.2 MB/s eta 0:00:02
   --- ------------------------------------ 0.8/8.6 MB 1.8 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/8.6 MB 1.5 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/8.6 MB 1.5 MB/s eta 0:00:05
   ------ --------------------------------- 1.3/8.6 MB 1.2 MB/s eta 0:00:07
   ------- -------------------------------- 1.6/8.6 MB 1.1 MB/s eta 0:00:07
   -------- -----------------------------

In [2]:

from ultralytics import YOLO
from roboflow import Roboflow
import matplotlib.pyplot as plt
import os

# Download dataset from Roboflow
rf = Roboflow(api_key="A2wycrFsDr1E3JqnNRlT")
project = rf.workspace("nur-byq0f").project("palm-detection-4qh3m")
version = project.version(2)
dataset = version.download("yolov8")

# Print dataset location to verify
print(f"Dataset downloaded to: {dataset.location}")



loading Roboflow workspace...
loading Roboflow project...
Dataset downloaded to: c:\Users\balak\PalmTreeDet\Palm-Detection-2


In [3]:
import os
from ultralytics import YOLO

# Memory optimization
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Load model (using 's' version for better accuracy than 'n' with minimal memory impact)
model = YOLO(r"C:\Users\balak\PalmTreeDet\runs\detect\train6\weights\best.pt")


results = model.train(
    # --- Core Training ---
    data=os.path.join(dataset.location, "data.yaml"),
    epochs=40,
    imgsz=512,        # Optimal for palm tree detection (balance between detail and memory)
    batch=6,          # Stable batch size for 4GB GPU
    device="0",
    amp=True,         # Mixed precision training (critical for 4GB GPUs)
    patience=20,      # Longer patience for palm detection
    
    # --- Palm-Specific Augmentations ---
    hsv_h=0.015,      # Slight hue variation (palms have consistent colors)
    hsv_s=0.4,        # Moderate saturation changes
    hsv_v=0.3,        # Limited value changes (avoid dark/bright extremes)
    fliplr=0.5,       # Keep horizontal flips (symmetrical trees)
    degrees=7.0,      # Small rotations (trees mostly upright)
    perspective=0.001, # Minimal perspective (aerial view consistency)
    
    # --- Memory Optimizations ---
    cache="ram",      # Faster if you have >16GB system RAM
    workers=2,        # Reduce if RAM issues occur
    optimizer="AdamW", # More memory efficient than SGD
    
    # --- Palm Detection Enhancements ---
    mixup=0.05,       # Helps with overlapping trees
    copy_paste=0.05,  # Simulates occluded palms
    overlap_mask=True, # Better for dense palm clusters
    mask_ratio=2,     # Optimizes mask processing
)

Ultralytics 8.3.106  Python-3.12.7 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=C:\Users\balak\PalmTreeDet\runs\detect\train6\weights\best.pt, data=c:\Users\balak\PalmTreeDet\Palm-Detection-2\data.yaml, epochs=40, time=None, patience=20, batch=6, imgsz=512, save=True, save_period=-1, cache=ram, device=0, workers=2, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=2, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
100%|██████████| 5.35M/5.35M [00:09<00:00, 623kB/s] 


AMP: checks passed 


train: Scanning C:\Users\balak\PalmTreeDet\Palm-Detection-2\train\labels.cache... 8532 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8532/8532 [00:00<?, ?it/s]


train: 9.4GB RAM required to cache images with 50% safety margin but only 3.0/15.4GB available, not caching images 


...

      34/40      1.44G       1.19     0.7728      1.558         19        512: 100%|██████████| 1422/1422 [02:47<00:00,  8.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:08<00:00,  8.37it/s]

                   all        813       1951      0.845      0.851      0.892      0.486
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 14, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



34 epochs completed in 1.694 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 22.5MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics 8.3.106  Python-3.12.7 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 68/68 [00:08<00:00,  8.27it/s]


                   all        813       1951      0.849       0.85      0.886      0.522
Speed: 0.3ms preprocess, 4.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train7


In [4]:
import torch
print(torch.cuda.is_available())       # Should be True
print(torch.cuda.get_device_name(0))   # Should say RTX 3050


True
NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
# 3. Enhanced Validation
metrics = model.val(
    conf=0.25,              # Confidence threshold
    iou=0.45,               # NMS IoU threshold
    plots=True,
 save=True,             # Generate plots
    save_json=True ,         # Save metrics to JSON
    visualize =True #show modified samples
)

# 4. Model Export (optional)
model.export(format='onnx')  # For production deployment

Ultralytics 8.3.106  Python-3.12.7 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\balak\PalmTreeDet\Palm-Detection-2\valid\labels.cache... 813 images, 0 backgrounds, 0 corrupt: 100%|██████████| 813/813 [00:00<?, ?it/s]


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.6GB RAM): 100%|██████████| 813/813 [00:01<00:00, 412.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 136/136 [00:10<00:00, 13.35it/s]


                   all        813       1951      0.857      0.864      0.879      0.556
Speed: 0.4ms preprocess, 7.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving runs\detect\train72\predictions.json...
Results saved to runs\detect\train72
Ultralytics 8.3.106  Python-3.12.7 torch-2.6.0+cu118 CPU (AMD Ryzen 7 4800H with Radeon Graphics)

PyTorch: starting from 'runs\detect\train7\weights\best.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) (1, 5, 5376) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ---------------------------------------- 14.5/14.5 MB 1.0 MB/s eta 0:00:0001
   -------------------------------------- 213.1/213.1 MB 568.5 kB/s eta 0:00:00

requirements: AutoUpdate success  246.0s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim', 'onnxruntime-gpu']
requirements:  Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx

'runs\\detect\\train7\\weights\\best.onnx'